# Node Embeddings

Here we will have a look at node embeddings and how to further reduce their dimensionality to be able to visualize them in a 2D plot. 

### Note about data dependencies

PageRank centrality and Leiden community are also fetched from the Graph and need to be calculated first.
This makes it easier to see if the embeddings approximate the structural information of the graph in the plot.
If these properties are missing you will only see black dots all of the same size.

<br>  

### References
- [jqassistant](https://jqassistant.org)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)
- [Tutorial: Applied Graph Embeddings](https://neo4j.com/developer/graph-data-science/applied-graph-embeddings)
- [Visualizing the embeddings in 2D](https://github.com/openai/openai-cookbook/blob/main/examples/Visualizing_embeddings_in_2D.ipynb)
- [Fast Random Projection](https://neo4j.com/docs/graph-data-science/current/machine-learning/node-embeddings/fastrp)
- [scikit-learn TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE)
- [AttributeError: 'list' object has no attribute 'shape'](https://bobbyhadz.com/blog/python-attributeerror-list-object-has-no-attribute-shape)

In [ ]:

# TODO In future it might make sense to also run a community detection algorithm co-located in here 
# to not depend on the order of execution. Ideally, this is only done when the data is missing.

In [ ]:
import os
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plot
import typing as typ
import numpy as np
from sklearn.manifold import TSNE
from neo4j import GraphDatabase

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The pandas version is {}.'.format(pd.__version__))


In [ ]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))
driver.verify_connectivity()

In [ ]:
def get_cypher_query_from_file(filename):
    with open(filename) as file:
        return ' '.join(file.readlines())

In [ ]:
def query_cypher_to_data_frame(filename, parameters_: typ.Optional[typ.Dict[str, typ.Any]] = None):
    records, summary, keys = driver.execute_query(get_cypher_query_from_file(filename),parameters_=parameters_)
    return pd.DataFrame([r.values() for r in records], columns=keys)

In [ ]:
def create_undirected_projection(parameters: dict) -> bool: 
    """
    Creates an undirected homogenous in-memory Graph projection for/with Neo4j Graph Data Science Plugin.
    It returns True if there is data available for the given parameter and False otherwise.
    Parameters
    ----------
    dependencies_projection : str
        The name prefix for the in-memory projection for dependencies. Example: "java-package-embeddings-notebook"
    dependencies_projection_node : str
        The label of the nodes that will be used for the projection. Example: "Package"
    dependencies_projection_weight_property : str
        The name of the node property that contains the dependency weight. Example: "weight25PercentInterfaces"
    dependencies_projection_embedding_dimension : str
        The number of the dimensions and therefore size of the resulting array of floating point numbers
    """
    
    is_data_missing=query_cypher_to_data_frame("../cypher/Dependencies_Projection/Dependencies_0_Check_Projectable.cypher", parameters).empty
    if is_data_missing: return False

    query_cypher_to_data_frame("../cypher/Dependencies_Projection/Dependencies_1_Delete_Projection.cypher", parameters)
    query_cypher_to_data_frame("../cypher/Dependencies_Projection/Dependencies_2_Delete_Subgraph.cypher", parameters)
    query_cypher_to_data_frame("../cypher/Dependencies_Projection/Dependencies_4_Create_Undirected_Projection.cypher", parameters)
    query_cypher_to_data_frame("../cypher/Dependencies_Projection/Dependencies_5_Create_Subgraph.cypher", parameters)
    return True

In [ ]:
def create_node_embeddings(cypher_file_name: str, parameters: dict) -> pd.DataFrame: 
    """
    Creates an in-memory Graph projection by calling "create_undirected_projection", 
    runs the cypher Query given as cypherFileName parameter to calculate and stream the node embeddings
    and returns a Dataframe with the results.
    
    cypher_file_name
    ----------
    Name of the file containing the Cypher query that executes node embeddings procedure.

    parameters
    ----------
    dependencies_projection : str
        The name prefix for the in-memory projection for dependencies. Example: "java-package-embeddings-notebook"
    dependencies_projection_node : str
        The label of the nodes that will be used for the projection. Example: "Package"
    dependencies_projection_weight_property : str
        The name of the node property that contains the dependency weight. Example: "weight25PercentInterfaces"
    dependencies_projection_embedding_dimension : str
        The number of the dimensions and therefore size of the resulting array of floating point numbers
    """
    
    is_data_available=create_undirected_projection(parameters)
    
    if not is_data_available:
        print("No projected data for node embeddings calculation available")
        empty_result = pd.DataFrame(columns=["codeUnitName", 'projectName', 'communityId', 'centrality', 'embedding'])
        return empty_result

    embeddings = query_cypher_to_data_frame(cypher_file_name, parameters)
    display(embeddings.head()) # Display the first entries of the table
    return embeddings

In [ ]:
def prepare_node_embeddings_for_2d_visualization(embeddings: pd.DataFrame) -> pd.DataFrame:
    if embeddings.empty: 
        print("No projected data for node embeddings dimensionality reduction available")
        return embeddings
    
    # Calling the fit_transform method just with a list doesn't seem to work (anymore?). 
    # It leads to an error with the following message: 'list' object has no attribute 'shape'
    # This can be solved by converting the list to a numpy array using np.array(..).
    # See https://bobbyhadz.com/blog/python-attributeerror-list-object-has-no-attribute-shape
    embeddings_as_numpy_array = np.array(embeddings.embedding.to_list())

    # Use t-distributed stochastic neighbor embedding (t-SNE) to reduce the dimensionality 
    # of the previously calculated node embeddings to 2 dimensions for visualization
    t_distributed_stochastic_neighbor_embedding = TSNE(n_components=2, verbose=1, random_state=50)
    two_dimension_node_embeddings = t_distributed_stochastic_neighbor_embedding.fit_transform(embeddings_as_numpy_array)
    display(two_dimension_node_embeddings.shape) # Display the shape of the t-SNE result

    # Create a new DataFrame with the results of the 2 dimensional node embeddings
    # and the code unit and artifact name of the query above as preparation for the plot
    node_embeddings_for_visualization = pd.DataFrame(data = {
        "codeUnit": embeddings.codeUnitName,
        "artifact": embeddings.artifactName,
        "communityId": embeddings.communityId,
        "centrality": embeddings.centrality,
        "x": [value[0] for value in two_dimension_node_embeddings],
        "y": [value[1] for value in two_dimension_node_embeddings]
    })
    display(node_embeddings_for_visualization.head()) # Display the first line of the results
    return node_embeddings_for_visualization
    

In [ ]:
def plot_2d_node_embeddings(node_embeddings_for_visualization: pd.DataFrame, title: str):
    if embeddings.empty:
        print("No projected data to plot available")
        return

    plot.scatter(
        x=node_embeddings_for_visualization.x,
        y=node_embeddings_for_visualization.y,
        s=node_embeddings_for_visualization.centrality * 200,
        c=node_embeddings_for_visualization.communityId,
        cmap=main_color_map,
    )
    plot.title(title)
    plot.show()

In [ ]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [ ]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

In [ ]:
# Main Colormap
main_color_map = 'nipy_spectral'

## 1. Java Packages

### 1.1 Generate Node Embeddings using Fast Random Projection (Fast RP) for Java Packages

[Fast Random Projection](https://neo4j.com/docs/graph-data-science/current/machine-learning/node-embeddings/fastrp) calculates an array of floats (length = embedding dimension) for every node in the graph. These numbers approximate the relationship and similarity information of each node and are called node embeddings. Random Projections is used to reduce the dimensionality of the node feature space while preserving pairwise distances.

The result can be used in machine learning as features approximating the graph structure. It can also be used to further reduce the dimensionality to visualize the graph in a 2D plot, as we will be doing here.

In [ ]:
java_package_embeddings_parameters={
    "dependencies_projection": "java-package-embeddings-notebook",
    "dependencies_projection_node": "Package",
    "dependencies_projection_weight_property": "weight25PercentInterfaces",
    "dependencies_projection_embedding_dimension":"64" 
}
embeddings = create_node_embeddings("../cypher/Node_Embeddings/Node_Embeddings_1d_Fast_Random_Projection_Stream.cypher", java_package_embeddings_parameters)


### 1.2 Dimensionality reduction with t-distributed stochastic neighbor embedding (t-SNE) for Java

This step takes the original node embeddings with a higher dimensionality (e.g. list of 32 floats) and
reduces them to a 2 dimensional array for visualization. 

> It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data.

(see https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE)

In [ ]:
node_embeddings_for_visualization = prepare_node_embeddings_for_2d_visualization(embeddings)

### 1.3 Plot the node embeddings reduced to two dimensions for Java

In [ ]:
plot_2d_node_embeddings(
    node_embeddings_for_visualization, 
    "Java Package nodes positioned by their dependency relationships using t-SNE"
)

## 1. Typescript Modules

### 1.1 Generate Node Embeddings using Fast Random Projection (Fast RP) for Typescript Modules

See section 1.2 for some background about node embeddings.

In [ ]:
typescript_module_embeddings_parameters={
    "dependencies_projection": "typescript-module-embeddings-notebook",
    "dependencies_projection_node": "Module",
    "dependencies_projection_weight_property": "lowCouplingElement25PercentWeight",
    "dependencies_projection_embedding_dimension":"32" 
}
embeddings = create_node_embeddings("../cypher/Node_Embeddings/Node_Embeddings_1d_Fast_Random_Projection_Stream.cypher", typescript_module_embeddings_parameters)


### 1.2 Dimensionality reduction with t-distributed stochastic neighbor embedding (t-SNE) for Typescript

This step takes the original node embeddings with a higher dimensionality (e.g. list of 32 floats) and
reduces them to a 2 dimensional array for visualization. 

> It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data.

(see https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE)

In [ ]:
node_embeddings_for_visualization = prepare_node_embeddings_for_2d_visualization(embeddings)

### 1.3 Plot the node embeddings reduced to two dimensions for Typescript

In [ ]:
plot_2d_node_embeddings(
    node_embeddings_for_visualization, 
    "Typescript Module nodes positioned by their dependency relationships using t-SNE"
)